In [1]:
# Imnporting packages
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
from selenium.webdriver.chrome.options import Options
from random import randint
from fake_useragent import UserAgent
from webdriver_manager.chrome import ChromeDriverManager
from bs4.element import Comment
import re
import numpy as np
import pandas as pd

In [2]:
# Setting up driver and its options
options = Options()
options.add_argument("window-size=1400,600")
ua = UserAgent()
a = ua.random
user_agent = ua.random
print(user_agent)
options.add_argument(f'user-agent={user_agent}')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430


Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2866.71 Safari/537.36




[WDM] - Driver [C:\Users\sbsha\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


In [3]:
driver.get('https://www.microsoft.com/en-us/diversity/inside-microsoft/cross-disability/hiring.aspx')
driver.title

'Inclusive Hiring at Microsoft'

In [4]:
# Importing links extracted from Bing
data = pd.read_csv('output_with_tldextract.csv')
data = data.rename(columns={'Company Name':'Company', 'Name of Web Page':'Name', 'Web Page Snippet':'Snippet'})
data

,Company,Query,URL,Name,Snippet,Domain Name
0,Microsoft,disabled employees at Microsoft,https://www.microsoft.com/en-us/diversity/insi...,Inclusive Hiring at Microsoft,Our annual Ability Summit brings Microsoft emp...,microsoft
1,Microsoft,disabled employees at Microsoft,https://blogs.microsoft.com/accessibility/empo...,Empowering employees with disabilities - Micro...,The Accenture/Disability:IN study last year sh...,microsoft
2,Microsoft,disabled employees at Microsoft,https://www.microsoft.com/en-us/diversity/insi...,Disability Hiring - microsoft.com,"About Disability Hiring At Microsoft, our miss...",microsoft
3,Microsoft,disabled employees at Microsoft,https://www.microsoft.com/en-us/diversity/insi...,Inside Microsoft,Disability at Microsoft Representing employees...,microsoft
4,Microsoft,disabled employees at Microsoft,https://news.microsoft.com/2000/07/19/microsof...,"Microsoft Recruits, Hires and Accommodates Peo...","Like any company, Microsoft walks a delicate l...",microsoft
...,...,...,...,...,...,...
390,Cisco,accommodation statement news Cisco,https://investor.cisco.com/news/news-details/2...,Investor Relations - Cisco Reports Fourth Quar...,Q4 Results: 1 Revenue : $13.4 billion Growth o...,cisco
391,Cisco,accommodation statement news Cisco,https://blogs.cisco.com/financialservices/what...,What will Cisco IMPACT unveil for Financial In...,Normally sellers from all over the world fly t...,cisco
392,Cisco,accommodation statement news Cisco,https://newsroom.cisco.com/feature-content?art...,Hotel Guests Checking Into Public Cisco TelePr...,"January 26, 2010. By James A. Martin. Hotels a...",cisco
393,Cisco,accommodation statement news Cisco,https://investor.cisco.com/news/news-details/2...,Investor Relations - UPDATE - New Cisco Study ...,"SAN JOSE, Calif. , Nov. 01, 2018 (GLOBE NEWSWI...",cisco


In [5]:
# Extracting paras from web pages
new = pd.DataFrame()
for i in range(len(data)):
    #sleep(2)
    print(i, data.URL[i])
    try:
        driver.get(data.URL[i])
        content = driver.page_source
        soup = BeautifulSoup(content, 'html.parser')
        res = []
        for a in soup.findAll('p'):
            if a.find('span'): 
                sres = ''
                for b in a.findAll('span'):
                    if not b.find('span'):
                        sres += ' ' + b.get_text(strip=True)
                res.append(sres)
            else:
                res.append(a.get_text(strip=True))
        #data.Text[i] = res
        title = driver.title
        length = len(res)
        company = [data.Company[i]]*length
        link = [data.URL[i]]*length
        name = [title]*length
        snippet = [data.Snippet[i]]*length
        query = [data.Query[i]]*length
        df = pd.DataFrame(list(zip(link, company, name, snippet, query, res)))
        new = new.append(df, ignore_index=True) 
    except Exception as e:
        print('Error')

0 https://www.microsoft.com/en-us/diversity/inside-microsoft/cross-disability/hiring.aspx
1 https://blogs.microsoft.com/accessibility/empowering-employees-with-disabilities/
2 https://www.microsoft.com/en-us/diversity/inside-microsoft/cross-disability/disabilityhiring
3 https://www.microsoft.com/en-us/diversity/inside-microsoft/default.aspx
4 https://news.microsoft.com/2000/07/19/microsoft-recruits-hires-and-accommodates-people-with-disabilities/
5 https://www.microsoft.com/en-us/Accessibility/disability-answer-desk
6 https://docs.microsoft.com/en-us/microsoft-365/admin/add-users/remove-former-employee
7 https://news.microsoft.com/features/leaders-who-revealed-disabilities-found-unexpected-rewards/
8 https://www.microsoft.com/en-us/diversity/programs/microsoftdisabilityscholarship.aspx
9 https://techcommunity.microsoft.com/t5/microsoft-forms-blog/ownership-transfer-of-your-employees-forms/ba-p/389244
10 https://news.microsoft.com/on-the-issues/2020/10/09/national-disability-employment-

86 https://support.microsoft.com/en-us/office/select-statement-56615a70-3458-4e7c-8580-bd1d19a5de91
87 https://docs.microsoft.com/en-us/office/vba/Language/Reference/User-Interface-Help/select-case-statement
88 https://docs.microsoft.com/en-us/office/client-developer/access/desktop-database-reference/select-statement-microsoft-access-sql
89 https://support.microsoft.com/en-us/office/create-procedure-statement-91c700d1-8076-4040-896a-a0b7cf9d9888
90 https://docs.microsoft.com/en-us/office/vba/access/concepts/miscellaneous/in-operator-microsoft-access-sql
91 https://answers.microsoft.com/en-us/msoffice/forum/msoffice_access-mso_other-mso_archive/using-if-statements-in-a-report-or-query/c35c26ed-c08c-4fce-960c-24751a84c435
92 https://careers.microsoft.com/us/en/accommodationrequest
93 https://www.microsoft.com/investor/reports/ar19/index.html
94 https://microsoftedgetips.microsoft.com/en-us/2/46
95 https://answers.microsoft.com/en-us/windows/forum/windows_10-windows_store/your-bank-couldn

190 https://www.cisco.com/c/dam/assets/cdc_content_elements/docs/csr_pdfweb_review/cisco_and_our_employees_section.pdf
191 https://community.cisco.com/t5/small-business-switches/disable-pnp-completely/td-p/4099096
192 https://community.cisco.com/t5/switching/switch-ports-disabled/td-p/1140220
193 https://news-blogs.cisco.com/emear/
194 https://community.cisco.com/t5/email-security/disable-tls-v1-0/td-p/2955088
195 https://meraki.cisco.com/blog/
196 https://community.cisco.com/t5/unified-computing-system/can-i-disable-a-vnic-or-vhba/td-p/1872234
197 https://newsroom.cisco.com/feature-content?articleId=2024547
198 https://learningnetwork.cisco.com/s/question/0D53i00000Kt2RP/trunk-port-is-getting-error-disabled
199 https://umbrella.cisco.com/blog/get-off-network-security-with-opendnss-roaming-client
200 https://www.cisco.com/c/en/us/about/accessibility/voluntary-product-accessibility-templates.html
201 https://www.cisco.com/c/en/us/td/docs/voice_ip_comm/cuipph/AccessibilityFeatures/8800-s

276 https://devblogs.microsoft.com/nuget/
277 https://azure.microsoft.com/en-us/blog/
278 https://cloudblogs.microsoft.com/industry-blog/financial-services/
279 https://techcommunity.microsoft.com/t5/microsoft-teams-blog/notifications-from-yammer-communities-are-now-available-in/ba-p/2080589
280 https://azure.microsoft.com/en-us/blog/topics/government/
281 https://cloudblogs.microsoft.com/dynamics365/it/
282 https://azure.microsoft.com/en-us/blog/topics/serverless/
283 https://news.microsoft.com/on-the-issues/2019/09/25/accessibility-supportability-anne-taylor/
284 https://techcommunity.microsoft.com/t5/microsoft-forms-blog/microsoft-forms-is-committed-to-inclusive-design-and-accessible/ba-p/257057
285 https://news.microsoft.com/category/press-releases/
286 https://www.microsoft.com/en-us/accessibility/supplier-toolkit-resources
287 https://news.microsoft.com/
288 https://docs.microsoft.com/en-us/compliance/regulatory/offering-WCAG-2-1
289 https://blogs.microsoft.com/accessibility/page

370 https://community.cisco.com/t5/network-security/can-t-access-certain-websites/td-p/2235915
371 https://community.cisco.com/t5/routing/access-list-remarks/td-p/1564711
372 https://investor.cisco.com/Home/default.aspx
373 http://newsroom.cisco.com/press-release-content?articleId=888280
374 https://community.cisco.com/t5/vpn/cisco-any-connect-no-access-to-lan/td-p/2595028
375 https://community.cisco.com/t5/custom/page/page-id/community-site-map?categoryId=technology-support
376 https://tools.cisco.com/security/center/resources/sql_injection
377 https://jobs.cisco.com/
378 https://newsroom.cisco.com/press-release-content?articleId=1918770
379 https://meeting-infohub.cisco.com/interop/
380 https://investor.cisco.com/news/news-details/2021/Cisco-Provides-Customers-Unmatched-Visibility-Across-Applications-and-the-Internet/default.aspx
381 https://transform.cisco.com/c/Exec_GoogleCloud?x=xlG4zk&lx=9DvHxH
382 https://transform.cisco.com/c/Techeng_GoogleCloud?x=Dz-PBD&lx=O5CA9d
383 https://n

In [8]:
# Creating a dataframe of paragraphs
new.columns = ['Link', 'Company', 'Page Name', 'Snippet', 'Query', 'Text']
# Removing paras with less than 3 words
new = new[new['Text'].apply(lambda x: len(x.split(' ')) > 2)]
# Resetting index
new = new.reset_index(drop=True)
new

,Link,Company,Page Name,Snippet,Query,Text
0,https://www.microsoft.com/en-us/diversity/insi...,Microsoft,Inclusive Hiring at Microsoft,Our annual Ability Summit brings Microsoft emp...,disabled employees at Microsoft,With the unemployment rate for people with dis...
1,https://www.microsoft.com/en-us/diversity/insi...,Microsoft,Inclusive Hiring at Microsoft,Our annual Ability Summit brings Microsoft emp...,disabled employees at Microsoft,"From the very first days of our company, Micro..."
2,https://www.microsoft.com/en-us/diversity/insi...,Microsoft,Inclusive Hiring at Microsoft,Our annual Ability Summit brings Microsoft emp...,disabled employees at Microsoft,"Recruit, onboard and development of Neurodiver..."
3,https://www.microsoft.com/en-us/diversity/insi...,Microsoft,Inclusive Hiring at Microsoft,Our annual Ability Summit brings Microsoft emp...,disabled employees at Microsoft,Microsoft is striving to promote an inclusive ...
4,https://www.microsoft.com/en-us/diversity/insi...,Microsoft,Inclusive Hiring at Microsoft,Our annual Ability Summit brings Microsoft emp...,disabled employees at Microsoft,The Disability ERG represents employees with d...
...,...,...,...,...,...,...
13333,https://sway.office.com/ZIetX05BXMPoBLHk,Microsoft,Our Disability Inclusion Journey,NaN,NaN,Health and Wellness
13334,https://sway.office.com/ZIetX05BXMPoBLHk,Microsoft,Our Disability Inclusion Journey,NaN,NaN,Team Centric Training
13335,https://sway.office.com/ZIetX05BXMPoBLHk,Microsoft,Our Disability Inclusion Journey,NaN,NaN,Company-wide Training
13336,https://sway.office.com/ZIetX05BXMPoBLHk,Microsoft,Our Disability Inclusion Journey,NaN,NaN,Employee Community


In [9]:
# Exporting dataframe to CSV files
new.to_csv('Paras1.csv', header=True, index=False)